## LLM experiments

In [1]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import ebooklib
from bs4 import BeautifulSoup
import re
from ebooklib import epub


import langchain
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

from langchain_community.llms import Ollama
#from langchain.embeddings.huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# import pypandoc
# pypandoc.download_pandoc()

### Load LLM Model

In [3]:
llm = Ollama(model="llama3.1")

### Epub loader

In [4]:
import os
mypath = '../data'

epub_files = [file for file in os.listdir(mypath) if file.endswith('.epub')]
epub_files

['allan-kardec-a-genese.epub',
 'allan-kardec-o-livro-dos-mediuns.epub',
 'allan-kardec-o-evangelho-segundo-o-espiritismo.epub',
 'allan-kardec-o-ceu-e-o-inferno.epub',
 'allan-kardec-o-que-e-espiritismo.epub',
 'allan-kardec-o-livro-dos-espiritos.epub']

In [5]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/michelarruda/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/michelarruda/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [6]:
from langchain_community.document_loaders import UnstructuredEPubLoader

data = []
for file_name in epub_files:
    print(file_name)
    loader = UnstructuredEPubLoader("../data/" + file_name)
    data.append(loader.load())

allan-kardec-a-genese.epub


[ERROR] Found /Users/michelarruda/Applications/pandoc/pandoc, but not using it because of an error:
Traceback (most recent call last):
  File "/Users/michelarruda/miniconda3/envs/spiritism-chat/lib/python3.10/site-packages/pypandoc/__init__.py", line 750, in _ensure_pandoc_path
    version_string = _get_pandoc_version(path)
  File "/Users/michelarruda/miniconda3/envs/spiritism-chat/lib/python3.10/site-packages/pypandoc/__init__.py", line 608, in _get_pandoc_version
    p = subprocess.Popen(
  File "/Users/michelarruda/miniconda3/envs/spiritism-chat/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/Users/michelarruda/miniconda3/envs/spiritism-chat/lib/python3.10/subprocess.py", line 1847, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename)
OSError: [Errno 86] Bad CPU type in executable: '/Users/michelarruda/Applications/pandoc/pandoc'


allan-kardec-o-livro-dos-mediuns.epub
allan-kardec-o-evangelho-segundo-o-espiritismo.epub
allan-kardec-o-ceu-e-o-inferno.epub
allan-kardec-o-que-e-espiritismo.epub
allan-kardec-o-livro-dos-espiritos.epub


In [7]:
list_concat = [item for sublist in data for item in sublist]


### Split documents

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(list_concat)

In [9]:
from langchain.prompts import PromptTemplate

# vectors
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",     # Provide the pre-trained model's path
)

persist_directory = "../vectodb"
vectorstore = Chroma.from_documents(collection_name="spiritism_db",
                                 documents=splits, embedding=embeddings,
                                 persist_directory=persist_directory)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
#prompt = hub.pull("rlm/rag-prompt")

template = """
You must answer just using the given context. Always answer in Portuguese
Context: {context}

Question: {question}
Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/Users/michelarruda/miniconda3/envs/spiritism-chat/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/michelarruda/miniconda3/envs/spiritism-chat/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


In [33]:
__persist_directory = "../vectodb"

# Load the existing Chroma DB from the persistence directory
__vectorstore = Chroma(collection_name="spiritism_db",
    persist_directory=persist_directory
)

# Retrieve and generate using the relevant snippets of the blog.
__retriever = vectorstore.as_retriever()

In [34]:
__retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x32f4e0e80>, search_kwargs={})

In [35]:

docs = __retriever.invoke("Espiritismo")

docs

[Document(metadata={'source': '../data/allan-kardec-o-livro-dos-espiritos.epub'}, page_content='apresentam a vantagem de ser perfeitamente inteligíveis, deixando ao\nvocábulo espiritualismo a acepção que lhe é própria. Diremos, pois, que\na doutrina espírita ou o Espiritismo tem por princípio as relações do\nmundo material com os Espíritos ou seres do mundo invisível. Os adeptos\ndo Espiritismo serão os espíritas, ou, se quiserem, os espiritistas.'),
 Document(metadata={'source': '../data/allan-kardec-o-livro-dos-mediuns.epub'}, page_content='passageiro, porque entravam a marcha do Espiritismo e porque os médiuns\npreferem fechar os olhos sobre o mérito do que lhes é comunicado. Esse\nponto exige grande estudo da parte dos espíritas esclarecidos e dos\nmédiuns, os quais devem concentrar toda a sua atenção para distinguir o\nverdadeiro do falso.”'),
 Document(metadata={'source': '../data/allan-kardec-o-livro-dos-mediuns.epub'}, page_content='Espiritista – Esta palavra, empregada a princ

In [13]:
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x32f4e0e80>, search_kwargs={}),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou must answer just using the given context. Always answer in Portuguese\nContext: {context}\n\nQuestion: {question}\nAnswer:\n')
| Ollama(model='llama3.1')
| StrOutputParser()

In [14]:
rag_chain.invoke("O que é o espiritismo?")

'A doutrina espírita ou o Espiritismo tem por princípio as relações do mundo material com os Espíritos ou seres do mundo invisível.'

In [15]:
rag_chain.invoke("Que ano foi criado o espiritismo?")

'Infelizmente, não há informações no contexto fornecido sobre a data de criação do espiritismo. O texto parece se concentrar mais na definição e características da doutrina, em vez de fornecer informações sobre sua história ou origens. Portanto, não posso fornecer uma resposta precisa.'

In [16]:
rag_chain.invoke("Qual a situação da alma imediatamente depois da morte do corpo?")

'Segundo Allan Kardec, a alma permanece ligada ao corpo por um elo frágil até que o coração pare de bater, sendo então que o Espírito pode ter recuperado sua lucidez e se considerar feliz por tê-lo deixado.'

In [17]:
rag_chain.invoke("Qual a situação da alma imediatamente depois da morte do corpo?")

'A situação da alma imediatamente depois da morte do corpo é que o desprendimento opera-se gradualmente, e o Espírito já pode ter recuperado a sua lucidez, tornando-se testemunha consciente da extinção da vida do corpo, considerando-se feliz por tê-lo deixado.'

In [18]:
rag_chain.invoke("O que é Vasco da Gama?")

'Infelizmente, não há nenhuma informação sobre Vasco da Gama no contexto fornecido. Você deseja continuar?'

In [19]:
rag_chain.invoke("Qual a situação da alma imediatamente depois da morte do corpo? Cite a referencia")

"A alma não está localizada num ponto particular do corpo; ela forma com o perispírito um todo fluídico, penetrável, assimilando-se ao corpo inteiro. Podemos figuradamente imaginar dois corpos semelhantes na forma, um encaixado no outro, confundidos durante a vida e separados depois da morte. Nessa ocasião um deles é destruído, enquanto o outro subsiste.\n\nReferência: Document(metadata={'source': '../data/allan-kardec-o-que-e-espiritismo.epub'})"

In [20]:
rag_chain.invoke("Qual a situação da alma imediatamente depois da morte do corpo? Cite qual livro encontrar essa informação")

'A situação da alma imediatamente depois da morte do corpo é descrever sua situação feliz ou desgraçada, as impressões e a transformação pela morte do corpo. Pode-se encontrar essa informação no "Livro dos Espíritos" de Allan Kardec.'

In [21]:
rag_chain.invoke("Qual a situação do espírito imediatamente depois do desencarne? Cite qual livro encontrar essa informação")

'Desculpe, mas não há informações sobre a situação de um espírito imediatamente após o desencarne no contexto fornecido. O contexto parece tratar principalmente de regras para comunicação com Espíritos e conceitos espiritas, mas não aborda diretamente a questão da vida após a morte ou do estado dos Espíritos logo após o desencarne.\n\nNo entanto, se procuramos por informações sobre a situação geral dos Espíritos, especialmente após o desencarne, uma menção é feita em "O Céu e o Inferno" (semelhante ao livro citado no contexto), mas não há uma passagem direta na qual seja discutido o estado imediato de um espírito após o desencarne.'

In [22]:
rag_chain.invoke("Qual a situação da pessoa imediatamente depois da morte do corpo? Para onde vai? Fica vagando pela terra? Cite qual livro encontrar essa informação")

'A situação da pessoa imediatamente depois da morte do corpo é que o Espírito não encontra explicação para a situação em que se acha. Ele crê não estar morto, por isso que se sente vivo; vê a um lado o corpo, sabe que lhe pertence, mas não compreende que esteja separado dele.\n\nEssa situação dura enquanto haja qualquer ligação entre o corpo e o perispírito. (Encontrada no livro "O Livro dos Espíritos")'

### Add history

In [23]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = Ollama(model="llama3.1")

# ### Construct retriever ###
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
You must answer just using the given context. \
Use the following pieces of retrieved context to answer the question. \
Always answer in Portuguese. \
If you don't know the answer, just say that you don't know. \
Use five sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

#### Session test 1

In [24]:
conversational_rag_chain.invoke(
    {"input": "O que é o espiritismo?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'O Espiritismo tem por princípio as relações do mundo material com os Espíritos ou seres do mundo invisível. Os adeptos deste ensinamento são conhecidos como espíritas ou espiritistas. A doutrina visa estabelecer uma conexão entre a vida terrena e o mundo espiritual. O Espiritismo é baseado na crença na existência da alma espiritual e imaterial, sendo a base de todas as religiões.'

In [25]:
conversational_rag_chain.invoke(
    {"input": "Qual tipo de predição? Cite um exemplo"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Peço desculpas, mas não há informações suficientes no contexto fornecido para responder à sua pergunta sobre o tipo de predição.'

In [26]:
conversational_rag_chain.invoke(
    {"input": "Você falou sobre predições feitas pelos espíritos, quais seriam essas?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Desculpe a falta de clareza anterior! As predições feitas pelos Espíritos mencionadas no contexto são aquelas que forem cumpridas ou sejam verificadas como verdadeiras. Um exemplo citado é o disseminador do Espiritismo vendo "cumprir-se o que fora predito", indicando que essas previsões são realizadas ao longo do tempo, confirmadas pela experiência e observação.'

In [27]:
conversational_rag_chain.invoke(
    {"input": 'Então, explique sobre "evocação antecipada" e "responder por antecipação"'},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'A "evocação antecipada" refere-se a um trabalho preparatório feito com calma antes da sessão de comunicação com os Espíritos. Esse processo predispõe o Espírito a responder e pode até incluir respostas por antecipação a perguntas, mostrando que o Espírito já as conhecia.'

In [28]:
conversational_rag_chain.invoke(
    {"input": 'As predições faladas não seria o conteúdo dos livros em si?'},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Peço desculpas, mas não há informações suficientes no contexto fornecido para responder à sua pergunta. O contexto menciona apenas a crença na existência da alma espiritual e imaterial e não discute predições de livros.'

In [29]:
conversational_rag_chain.invoke(
    {"input": 'Você falou sobre predições feitas pelos espíritos, essas predições não seria o conteúdo dos livros em si?'},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Peço desculpas, mas não há informações suficientes no contexto fornecido para responder à sua pergunta com precisão. As conversas anteriormente mencionadas discutiam conceitos gerais do Espiritismo e suas crenças básicas.'

In [30]:
conversational_rag_chain.invoke(
    {"input": 'Você falou sobre predições feitas pelos espíritos, essas predições não seria o conteúdo dos livros em si?'},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Desculpe, mas não há informações suficientes no contexto fornecido para responder à sua pergunta com precisão. As conversas anteriormente mencionadas discutiam conceitos gerais do Espiritismo e suas crenças básicas.'

#### Session test 2

In [31]:
conversational_rag_chain.invoke(
    {"input": 'Como funciona o desenlace do corpo material como espírito após o desencarne?'},
    config={"configurable": {"session_id": "2"}},
)["answer"]

'O Espírito sobrevive à destruição do corpo material e retorna ao mundo espiritual. Privado do Espírito, o corpo é apenas matéria inerte. Sem o corpo, o Espírito é a vida e a inteligência. Em deixando o corpo, ele torna a existir no mundo espiritual, para depois reencarnar. O Espírito é o ser principal, racional e inteligente, enquanto o corpo é apenas um invólucro material temporário.'

In [32]:
conversational_rag_chain.invoke(
    {"input": 'No livro, é explicado como funciona o processo de desenlace do corpo material como espírito após a morte. Como funciona?'},
    config={"configurable": {"session_id": "2"}},
)["answer"]

KeyboardInterrupt: 

In [ ]:
conversational_rag_chain.invoke(
    {"input": 'Gostaria de saber um pouco sobre a influência de espíritos sobre nossos pensamentos'},
    config={"configurable": {"session_id": "2"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": 'Existe uma influencia oculta dos espíritos sobre os nossos pensamentos e sobre nossas ações, fale um pouco sobre isso'},
    config={"configurable": {"session_id": "2"}},
)["answer"]

#### Session test 3

In [ ]:
conversational_rag_chain.invoke(
    {"input": 'Segundo o contexto, explique sobre possessão'},
    config={"configurable": {"session_id": "3"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": 'Para isso, o exorcismo funciona?'},
    config={"configurable": {"session_id": "3"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": 'Mas o contexto diz algo sobre o exorcísmo?'},
    config={"configurable": {"session_id": "3"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": 'Entendi. E a prece? Ajuda a expulsar os maus espíritos?'},
    config={"configurable": {"session_id": "3"}},
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": 'Mas eu gostaria de saber sobre a prece na possessão de pessoas'},
    config={"configurable": {"session_id": "3"}},
)["answer"]